# 12.16 课后作业： 股票时间序列分析
------

本次作业将使用线性自回归模型和简单的神经网络模型对某公司每日收盘价格序列进行分析和预测。

In [ ]:
import csv, time
from tqdm import tqdm
import numpy as np
import torch
from torch import nn
from matplotlib import pyplot as plt

In [ ]:
filename = 'GOOGL_2006-01-01_to_2018-01-01.csv'
# skip the header. extract the 4th column ("close")
data = torch.from_numpy(np.loadtxt(filename, skiprows=1, delimiter=",", usecols=4)).float()
Len = len(data)
print("Totally", Len, "time steps.")

在$K$阶的自回归模型中，我们对时间序列建模如下：对任意$t$，
$$
x_{t} = f_\theta(x_{t-K}, x_{t-K+1}, ..., x_{t-1}) + \epsilon_{t},
$$
其中$\epsilon_t \sim \mathcal{N}(0, \sigma^2)$.

为了求出最优的模型参数$\theta$，我们求解这样的优化问题：
$$
\min_{\theta} \mathcal{L}_\text{train}(\theta) := \sum_{t\in \mathcal{I}_{\text{train}}} \Vert f_\theta(x_{t-K}, x_{t-K+1}, ..., x_{t-1}) - x_t \Vert_2^2
$$

在线性自回归模型中，我们设$f_\theta(x_1, ..., x_K) = w_0 + \sum_{k=1}^K w_kx_k$是一个线性模型，其中参数$\theta = \{w_0, w_1, ..., w_K\}$；而在神经网络模型中，$f_\theta$可以拟合更加复杂的映射。

-----
下面首先定义一个`Pytorch`中的标准数据集类，这是个可以随机访问和迭代的对象。

In [ ]:
class StockDataset(torch.utils.data.Dataset):
    
    def __init__(self, data, K):
        self.data = data
        self.K = K
    
    def __len__(self):
        return len(self.data) - self.K
    
    def __getitem__(self, idx): # return the `idx` - th object (x, y) in the dataset
        return ( self.data[idx : idx + K], 
                self.data[idx + K].unsqueeze(0) )# insert a new axis

## 1. 线性自回归模型

In [ ]:
K = 30

In [ ]:
train_data = StockDataset(data[:-500], K)
test_data = StockDataset(data[-500:], K)

In [ ]:
# train_data 可迭代
for x, y in train_data:
    print(x, y)
    break

训练集的$X \in \mathbb{R}^{N \times (K+1)}$和$y \in \mathbb{R}^{N\times 1}$。$X$的每一行表示一个$K+1$维数据点（新加的一维用来把常数项放进求和号中，简化表达），各个点的回归目标保存在$y$中。

In [ ]:
X = torch.cat((torch.stack([x for x, _ in train_data]), torch.ones(len(train_data), 1)), axis = 1)
y = torch.stack([y for _, y in train_data])

设线性模型参数$w\in \mathbb{R}^{K+1}$，则正规方程$\min_w \Vert Xw - y \Vert_2^2$的解为$\hat{w} = (X^TX)^{-1}X^Ty$. 

**请你在下面补充代码，求出最优的参数$\hat{w}$。**

In [ ]:
W = 

**请你在下面补充代码，检查求出的模型在测试集`test_data`上的表现，求出测试集上的平均均方误差。**

In [ ]:
X_test = 
y_test = 
loss = 
print(loss)

**计算模型在整个序列上的估计值，并可视化。** 结合系数`w`，你发现了什么现象，如何解释模型的行为？

In [ ]:
y_hat = 

plt.plot(y_hat)
plt.plot(data)

In [ ]:
W

**调整窗口宽度$K$的大小，对比Loss，你有什么发现？**

## 2. 神经网络模型

在这里，我们设计一个简单的两到三层的神经网络来预测时间序列。

首先将训练集进一步抽出一部分作为验证集。

In [ ]:
train_size = int(0.9 * len(train_data))
valid_size = len(train_data) - train_size
train_data, valid_data = torch.utils.data.random_split(train_data, [train_size, valid_size])

构造`Pyrorch`中标准的`DataLoader`类。你可以更改其中的参数。

In [ ]:
train_iter  = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size = 64)
valid_iter = torch.utils.data.DataLoader(valid_data, batch_size = 64)
test_iter = torch.utils.data.DataLoader(test_data, batch_size = 64)

**定义模型及损失函数。** 提示：为了加速训练，你可以考虑在forward中，先对网络的输入（每个长度为K的片段）进行归一化。

In [ ]:
class MyModel(nn.Module):
    def __init__(self, ...):
        
        
    def forward(self, X):
        

net = 
print(net)

In [ ]:
loss = nn.MSELoss()

训练模型。`evaluate_model`用于评估模型`net`在`data_iter`上的表现，给出平均损失；`train`会在`train_iter`上训练模型`net`，优化器选择`optimizer`，并用`valid_iter`进行验证及early stop。

**请你完善以下代码。**代码仅供参考，觉得不方便的地方，可以直接修改。

提示：如果你是用cuda进行训练，请务必把网络和需要在GPU上进行运算的tensor移动到GPU上。参与运算的两个tensor必须同时在GPU显存或CPU内存中。

In [ ]:
from utils import EarlyStop

device = "cuda" if torch.cuda.is_available() else "cpu"

def evaluate_model(data_iter, net, loss, device = device):
    loss_sum, n = 0.0, 0
    net = net.to(device)
    with torch.no_grad():
        net.eval() 
        for X, y in data_iter:
            X = X.to(device)
            y = y.to(device)
            
            
        net.train()
    return loss_sum / n

def train(net, train_iter, valid_iter, loss, optimizer, max_epochs = 100, early_stop = None, device = device):
    
    net = net.to(device)
    print("training on ", device)
    
    for epoch in range(max_epochs):
        
        train_loss, n, start = 0.0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            
            

            train_loss += l.cpu().item() * y.shape[0]
            n += y.shape[0]
            
        valid_loss = 
        
        print('epoch %d, train loss %.8f , valid loss %.8f, time %.1f sec' % (epoch, train_loss, valid_loss, time.time() - start))
        
        if (early_stop):
            if (early_stop(valid_loss, net, optimizer)):
                break
    
    if (early_stop):
        checkpoint = torch.load(early_stop.save_name)
        net.load_state_dict(checkpoint["net"])

In [ ]:
train(net, train_iter, valid_iter, loss, 
      optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=  , weight_decay = 0.0001), 
      max_epochs = 300, early_stop = EarlyStop(patience = 10))

In [ ]:
print("test loss: ", evaluate_model(test_iter, net, loss))

**哪个模型的表现更好？你有哪些思考？**

另外，你也可以尝试别的模型，如上课提到的像WaveNet那样进一步堆叠网络。你可以另附代码或直接修改上面的代码。